# Equity betas

In [1]:
import pandas as pd
import pandas_datareader as web
import numpy as np
import datetime as dt
import statsmodels.api as sm

### Calculate the 5-yr montly betas of Korean Airlines, Qantas Airways, ANA Holdings and Singapore Airlines 

In [2]:
start_date = dt.datetime(2016, 12, 31)
end_date = dt.datetime(2019, 12, 31)

tickers = ['APRN', 'GRUB', 'DHER.DE', 'HFG.DE', '^GSPC', '^GDAXI']

In [3]:
# 10 years adjusted close price of GB Group and FTSE100
df = pd.DataFrame()

for ticker in tickers:
    df[ticker] = web.DataReader(ticker, 'yahoo', start_date, end_date)['Close']

df

,APRN,GRUB,DHER.DE,HFG.DE,^GSPC,^GDAXI
Date,,,,,,
2017-06-29,150.000000,43.200001,NaN,NaN,2419.699951,12416.190430
2017-06-30,140.100006,43.599998,27.799999,NaN,2423.409912,12325.120117
2017-07-03,145.050003,43.410000,27.500000,NaN,2429.010010,12475.309570
2017-07-05,132.600006,43.459999,25.549999,NaN,2432.540039,12453.679688
2017-07-06,120.900002,43.130001,25.900000,NaN,2409.750000,12381.250000
...,...,...,...,...,...,...
2019-12-24,7.670000,48.580002,NaN,NaN,3223.379883,NaN
2019-12-26,7.470000,49.060001,NaN,NaN,3239.909912,NaN
2019-12-27,7.150000,49.060001,70.660004,18.82,3240.020020,13337.110352


In [4]:
df = df.resample('W').mean()
df.head()

,APRN,GRUB,DHER.DE,HFG.DE,^GSPC,^GDAXI
Date,,,,,,
2017-07-02,145.050003,43.400000,27.799999,NaN,2421.554932,12370.655273
2017-07-09,128.625002,43.420000,26.300000,NaN,2424.119995,12424.729736
2017-07-16,112.800000,44.786000,27.510000,NaN,2440.662012,12556.513867
2017-07-23,98.010001,45.517999,28.889000,NaN,2467.914014,12431.381836
2017-07-30,105.360001,46.344000,27.573000,NaN,2474.477979,12230.622070


In [5]:
returns = df.copy()
for x in returns.columns:
    returns[x] = np.log(df[x]).diff()

returns.dropna(inplace=True)
returns

,APRN,GRUB,DHER.DE,HFG.DE,^GSPC,^GDAXI
Date,,,,,,
2017-11-12,-0.296798,0.025865,0.008151,-0.103470,0.003710,-0.007159
2017-11-19,-0.066249,0.020177,0.002252,-0.005056,-0.003890,-0.021499
2017-11-26,-0.013463,0.049626,0.025808,0.078332,0.006418,0.003849
2017-12-03,0.013463,0.006113,-0.002395,0.003997,0.012902,-0.005665
2017-12-10,0.208291,0.008623,-0.048716,-0.035652,0.003223,0.004569
...,...,...,...,...,...,...
2019-12-08,0.004422,-0.025304,0.036659,0.001575,-0.008167,-0.014319
2019-12-15,0.025840,-0.018947,0.071938,0.033728,0.010489,0.007801
2019-12-22,0.072407,0.135889,0.245199,-0.032643,0.016015,0.009391


market_return = 0.0812
risk_free = 0.035

returns['MRP'] = returns['S&P500'] - risk_free
returns.dropna(inplace=True)
returns

In [6]:
X = returns['^GSPC']
X1 = sm.add_constant(X)
betas = []
for i in returns.columns[:2]:
    y = returns[i]
    model = sm.OLS(y, X1)
    results = model.fit()
    betas.append(results.params['^GSPC'])

X_GER = returns['^GDAXI']
X_GER1 = sm.add_constant(X_GER)

for i in returns.columns[2:4]:       
    y_GER = returns[i]
    model2 = sm.OLS(y_GER, X_GER1)
    results2 = model2.fit()
    betas.append(results2.params['^GDAXI'])

betas

[1.6349563675018808,
 1.4935593282414734,
 0.8096076762565098,
 0.8342137439882905]

In [7]:
equity_betas = pd.DataFrame(betas, index=returns.columns[:-2])

equity_betas.T

,APRN,GRUB,DHER.DE,HFG.DE
0,1.634956,1.493559,0.809608,0.834214


In [8]:
returns.columns[2: 4]

Index(['DHER.DE', 'HFG.DE'], dtype='object')